# Create for study about subjects the exam


### 1) Introdução a correlação.

* Interesse em analisar o comportamento de duas variáveis quantitativas.(causa -> efeito)
* Interesse em obter uma medida estatística que indique se existe ou não associação linear entre duas variáveis; e se existe, qual a sua magnitude e sinal.
* A correlação entre duas variáveis é medida através da seguinte fómula.
    * $$
        r = \frac{\sum_{i=1}^{n} X_i Y_i - n\bar{x}\bar{y}}
        {\sqrt{\sum_{i=1}^{n} X_i^2 - n(\bar{x})^2} \sqrt{\sum_{i=1}^{n} Y_i^2 - n(\bar{y})^2}}
    $$
* No exemplo abaixo podemos observar correlações bem fortes (> 0.7 ou <-0.7) uma correlação média(|>0.3 && <0.7 | ECOR3) e duas correlações fracas( BEEF3, ABEV3 )

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
import re
import plotly.express as px


# Credenciais do banco de dados (ajuste se necessário)
DB_USER = "user_invest"
DB_PASSWORD = "senha_forte"
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'postgres'
TABLE_NAME = 'companys'
tickers_list = ["QUAL3.SA", "DASA3.SA", "HAPV3.SA", "FLRY3.SA", "ODPV3.SA", "RDOR3.SA",
    "MATD3.SA", "GNDI3.SA", "PFRM3.SA", "RADL3.SA", "HYPE3.SA", "BLAU3.SA",
    "ONCO3.SA", "AALR3.SA",
    "CCRO3.SA", "ECOR3.SA", "SBSP3.SA", "RAIL3.SA", "RENT3.SA",
    "ELET3.SA", "ENGI11.SA", "EQTL3.SA", "EGIE3.SA", "CMIG4.SA", "CPLE6.SA",
    "NEOE3.SA", "CPFE3.SA", "TAEE11.SA", "ENEV3.SA", "ISAE4.SA", "AURE3.SA",
    "AESB3.SA", "LIGT3.SA",
    "BRFS3.SA", "BEEF3.SA", "MRFG3.SA", "JBSS3.SA", "ABEV3.SA",
    "CAML3.SA", "MDIA3.SA", "SMTO3.SA",
    "^BVSP"
    ]

try:
#Cria conexão com o banco de dados postgres
    db_url = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
    engine = create_engine(db_url)
    query = f"""
        SELECT data, ticker, "Fechamento" 
        FROM companys 
        WHERE ticker IN ({', '.join([f"'{t}'" for t in tickers_list])}) 
        ORDER BY data;
        """
    df = pd.read_sql(query, engine)

    df['ticker'] = df['ticker'].apply(lambda x: re.sub(r'(\.SA)$', '',x).replace('^', ''))
    #Reorganizando o dataframe para que cada ticker seja uma coluna.
    df_pivot = df.pivot_table(index='data', columns='ticker', values='Fechamento')
    #calculando as correlações das matrix.
    correlation_matrix = df_pivot.corr()

    #Pega a correlação do Ibovespa com as outras ações e ordena
    ibov_correlations = correlation_matrix['BVSP'].drop('BVSP').sort_values()
    df_plot = ibov_correlations.reset_index()
    df_plot.columns = ['Ticker', 'Correlacao']
    # Calcula a altura do gráfico com base no número de tickers
    # Isso garante que haja espaço suficiente para todos os rótulos
    plot_height = len(df_plot) * 25 + 150 
    
    # Cria o gráfico de barras dinâmico com Plotly
    fig = px.bar(
        df_plot,
        x='Correlacao',
        y='Ticker',
        orientation='h',
        title='Correlação entre o Ibovespa e Outras Ações (Fechamento)',
        labels={'Correlacao': 'Coeficiente de Correlação', 'Ticker': 'Ações'},
        color='Correlacao',
        color_continuous_scale=px.colors.sequential.Rainbow,
        hover_data={'Correlacao': ':.3f'},
        height=plot_height # Adiciona a altura calculada
    )

    # Remove o ajuste automático de texto que pode ocultar rótulos
    fig.update_layout(uniformtext_minsize=8) 

    # Exibe o gráfico diretamente na célula do Jupyter
    fig.show()
    print("Você pode interagir com o gráfico e passar o mouse sobre as barras para ver os valores.")



except Exception as e:
    print(f"Ocorreu um erro: {e}")

Você pode interagir com o gráfico e passar o mouse sobre as barras para ver os valores.


### 2) Intervalo de Confiança e Teste de Hipóteses para Correlação.
*   O coeficiente de correlação populacional é um parâmetro ou característica da população, representada pela letra grega $\rho$ e desconhecido. 
*   Dada uma amostra simples do par de variáveis aleatórias (X,Y), o coeficiente r pode ser considerado uma estimativo do verdadeiro e desconhecido coeficiente $\rho$ .
*   Podemos usar o coeficiente de correlação amostral , r , para fazer várias inferencias sobre $\rh$ .
*   Uma população que tenha duas variáveis não correlacionadas, pode produzir uma amostra com coeficiente de relação diferente de 0, simplesmente deviado a seleção dos dados.


In [2]:
import numpy as np
from scipy import stats
import pandas as pd
from sqlalchemy import create_engine
import re
import plotly.express as px

# --- FUNÇÃO DE CÁLCULO DE IC (Transformação de Fisher) ---
def calcular_ic_correlacao(r, n, nivel_confianca=0.95):
    """Calcula o IC para o coeficiente de correlação usando a Transformação de Fisher."""
    
    # Limita r para evitar erros em np.arctanh se for exatamente 1 ou -1
    r_clipped = np.clip(r, -0.99999999, 0.99999999) 
    
    # n precisa ser pelo menos 4 para que n-3 seja >= 1
    if n <= 3:
        return np.nan, np.nan 

    # 1. Transformação de Fisher (r para z)
    z = np.arctanh(r_clipped)
    
    # 2. Erro Padrão de z e Valor Z crítico
    se_z = 1 / np.sqrt(n - 3)
    alpha = 1 - nivel_confianca
    z_critico = stats.norm.ppf(1 - alpha/2)
    
    # 3. Limites do IC na escala Z
    z_lower = z - z_critico * se_z
    z_upper = z + z_critico * se_z
    
    # 4. Transformação inversa (Z de volta para r)
    r_lower = np.tanh(z_lower)
    r_upper = np.tanh(z_upper)
    
    return r_lower, r_upper
# --------------------------------------------------------

# --- CONFIGURAÇÕES E CONEXÃO (Se necessário, mantenha no topo do notebook) ---
DB_USER = "user_invest"
DB_PASSWORD = "senha_forte"
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'postgres'
TABLE_NAME = 'companys'
IBOV_TICKER = 'BVSP'

tickers_list = ["QUAL3.SA", "DASA3.SA", "HAPV3.SA", "FLRY3.SA", "ODPV3.SA", "RDOR3.SA",
    "MATD3.SA", "GNDI3.SA", "PFRM3.SA", "RADL3.SA", "HYPE3.SA", "BLAU3.SA",
    "ONCO3.SA", "AALR3.SA",
    "CCRO3.SA", "ECOR3.SA", "SBSP3.SA", "RAIL3.SA", "RENT3.SA",
    "ELET3.SA", "ENGI11.SA", "EQTL3.SA", "EGIE3.SA", "CMIG4.SA", "CPLE6.SA",
    "NEOE3.SA", "CPFE3.SA", "TAEE11.SA", "ENEV3.SA", "ISAE4.SA", "AURE3.SA",
    "AESB3.SA", "LIGT3.SA",
    "BRFS3.SA", "BEEF3.SA", "MRFG3.SA", "JBSS3.SA", "ABEV3.SA",
    "CAML3.SA", "MDIA3.SA", "SMTO3.SA",
    "^BVSP"
    ]

try:
    # --- CONEXÃO E CARREGAMENTO DE DADOS (Início da Lógica da Célula 13) ---
    db_url = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
    engine = create_engine(db_url)
    
    # 1. Carregar e pivotar os dados
    tickers_sql_list = ", ".join([f"'{t}'" for t in tickers_list])
    query = f"""
        SELECT data, ticker, "Fechamento" 
        FROM {TABLE_NAME} 
        WHERE ticker IN ({tickers_sql_list}) 
        ORDER BY data;
        """
    df = pd.read_sql(query, engine)

    # Limpeza dos nomes dos tickers
    df['ticker'] = df['ticker'].apply(lambda x: re.sub(r'(\\.SA)$', '',x).replace('^', ''))
    df_pivot = df.pivot_table(index='data', columns='ticker', values='Fechamento')
    
    OUTRAS_ACOES = [t for t in df_pivot.columns if t != IBOV_TICKER]
    N_OBSERVACOES_TOTAL = len(df_pivot)

    resultados_correlacao = []
    
    # 2. Loop para calcular as estatísticas por par
    for ticker in OUTRAS_ACOES:
        # Cria um DataFrame temporário para alinhar e remover NaNs em pares
        df_temp = pd.DataFrame({'acao': df_pivot[ticker], 'ibov': df_pivot[IBOV_TICKER]}).dropna()
        
        serie_acao_limpa = df_temp['acao']
        serie_ibov_limpa = df_temp['ibov']
        n_valido = len(serie_acao_limpa)
        
        # 3. Verificação de dados: N suficiente e não variância zero
        # Se n < 5 (ou 4 para o IC), ou se a variação de um dos ativos for zero (std() == 0),
        # o cálculo da correlação e do IC não é possível ou é inválido.
        if n_valido < 5 or serie_acao_limpa.std() == 0 or serie_ibov_limpa.std() == 0:
            resultados_correlacao.append({
                'Ticker': ticker,
                'Correlacao': np.nan,
                'P_Valor': np.nan,
                'IC_Inferior': np.nan,
                'IC_Superior': np.nan,
                'N': n_valido
            })
            continue
        
        # 4. Correlação de Pearson e p-valor
        r, p_value = stats.pearsonr(serie_acao_limpa, serie_ibov_limpa)
        
        # 5. Intervalo de Confiança
        ic_inferior, ic_superior = calcular_ic_correlacao(r, n_valido, nivel_confianca=0.95)
        
        resultados_correlacao.append({
            'Ticker': ticker,
            'Correlacao': r,
            'P_Valor': p_value,
            'IC_Inferior': ic_inferior,
            'IC_Superior': ic_superior,
            'N': n_valido
        })
    
    # 6. Criar o DataFrame final e ordenar
    df_resultados = pd.DataFrame(resultados_correlacao).sort_values(by='Correlacao', ascending=False).reset_index(drop=True)
    
    # 7. Adicionar coluna de significância
    df_resultados['Significancia'] = df_resultados['P_Valor'].apply(
        lambda p: 'Significativo (p < 0.05)' if p < 0.05 else 'Não Significativo (p ≥ 0.05)'
    )
    
    # ----------------------------------------------------------------------------------
    # EXIBIÇÃO DA TABELA E GRÁFICO (como no código original)
    # ----------------------------------------------------------------------------------
    
    print("--- Tabela de Resultados Estatísticos ---\n")
    print(df_resultados[['Ticker', 'Correlacao', 'IC_Inferior', 'IC_Superior', 'P_Valor', 'Significancia']])
    
    plot_height = len(df_resultados) * 25 + 150 
        
    fig = px.bar(
        df_resultados,
        x='Correlacao',
        y='Ticker',
        orientation='h',
        color='Significancia',
        color_discrete_map={
            'Significativo (p < 0.05)': 'royalblue', 
            'Não Significativo (p ≥ 0.05)': 'lightcoral'
        },
        title=f'Correlação (Pearson) com o Ibovespa (N={N_OBSERVACOES_TOTAL} dias)',
        labels={'Correlacao': 'Coeficiente de Correlação (r)', 'Ticker': 'Ações'},
        hover_data={
            'Correlacao': ':.3f', 
            'P_Valor': ':.4f',
            'IC_Inferior': ':.3f',
            'IC_Superior': ':.3f',
            'N': True
        },
        height=plot_height
    )
    
    fig.update_layout(uniformtext_minsize=8)
    fig.show()
    
    print("\nPrimeiras linhas da coluna BVSP:")
    print(df_pivot['BVSP'].head())
    
    # Verificação 2: Quantos valores válidos (não-NaN) tem o BVSP?
    print("\nContagem de valores válidos (non-NaN) no BVSP:")
    print(df_pivot['BVSP'].count())
    
    # Verificação 3: Onde estão os NaNs? (Mostra o total de NaNs por ticker)
    print("\nTotal de NaNs por Ticker no df_pivot:")
    print(df_pivot.isnull().sum().sort_values(ascending=False))

except Exception as e:
    print(f"Ocorreu um erro: {e}")

--- Tabela de Resultados Estatísticos ---

      Ticker  Correlacao  IC_Inferior  IC_Superior       P_Valor  \
0   EGIE3.SA    0.936927     0.931931     0.941567  0.000000e+00   
1   ELET3.SA    0.934240     0.929039     0.939072  0.000000e+00   
2   CPFE3.SA    0.873056     0.863358     0.882109  0.000000e+00   
3   ENEV3.SA    0.846845     0.835321     0.857625  0.000000e+00   
4   SBSP3.SA    0.829828     0.817150     0.841704  0.000000e+00   
5   CMIG4.SA    0.821008     0.807742     0.833443  0.000000e+00   
6   SMTO3.SA    0.798159     0.783401     0.812018  0.000000e+00   
7   MRFG3.SA    0.707807     0.687590     0.726927  0.000000e+00   
8   ECOR3.SA    0.307114     0.271007     0.342357  2.972592e-55   
9   BEEF3.SA    0.093764     0.054581     0.132658  2.946389e-06   
10  ABEV3.SA   -0.164549    -0.202620    -0.125981  1.698918e-16   
11  BRFS3.SA   -0.792593    -0.806792    -0.777478  0.000000e+00   

               Significancia  
0   Significativo (p < 0.05)  
1   Signif


Primeiras linhas da coluna BVSP:
data
2015-01-05    47517.0
2015-01-06    48001.0
2015-01-07    49463.0
2015-01-08    49943.0
2015-01-09    48840.0
Name: BVSP, dtype: float64

Contagem de valores válidos (non-NaN) no BVSP:
2477

Total de NaNs por Ticker no df_pivot:
ticker
BVSP        8
ABEV3.SA    0
BEEF3.SA    0
BRFS3.SA    0
CMIG4.SA    0
CPFE3.SA    0
ECOR3.SA    0
EGIE3.SA    0
ELET3.SA    0
ENEV3.SA    0
MRFG3.SA    0
SBSP3.SA    0
SMTO3.SA    0
dtype: int64
